In [ ]:
"""

Prepare the masks of the ice shelves based on BedMachine

"""

In [ ]:
import xarray as xr
import numpy as np
from pyproj import Transformer
import pandas as pd
from tqdm.notebook import trange, tqdm
import cc3d
import matplotlib as mpl


import multimelt.plume_functions as pf
import multimelt.box_functions as bf
import multimelt.useful_functions as uf
import basal_melt_param.create_isf_mask_functions as isfmf

import distributed

In [ ]:
%matplotlib qt5

In [ ]:
map_lim = [-3000000,3000000]

#chunk_size = 700
chunk_size = False

READ IN DATA

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
inputpath_metadata='/bettik/burgardc/SCRIPTS/basal_melt_param/data/raw/MASK_METADATA/'
outputpath_mask ='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
outputpath_boxes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/BOXES/BedMachine_4km/'
outputpath_plumes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/PLUMES/BedMachine_4km/'

file_mask_orig = xr.open_dataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
file_mask_orig_cut = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim)

In [ ]:
file_msk = file_mask_orig_cut['mask_0_1_2']  #0 = ocean, 1 = ice shelves, 2 = grounded ice

In [ ]:
file_bed_orig = -1*file_mask_orig_cut['bed']
file_draft = (file_mask_orig_cut['thickness'] - file_mask_orig_cut['surface']).where(file_msk==1)
file_draft = file_draft.where(file_draft > 0, 0)
file_isf_conc = file_mask_orig_cut['isf_conc']
file_msk = file_mask_orig_cut['mask_0_1_2']  #0 = ocean, 1 = ice shelves, 2 = grounded ice

xx = file_msk['x']
yy = file_msk['y']

dx = abs(xx[1] - xx[0])
dy = abs(yy[1] - yy[0])

In [ ]:
IMBIE_file = xr.open_dataset(inputpath_data + 'Mask_Iceshelf_4km_IMBIE_withNisf.nc')

In [ ]:
IMBIE_file['Iceshelf_extrap'].where(IMBIE_file['Iceshelf_extrap'] == 2).plot()

In [ ]:
prelim_file = file_mask_basins['ID_IMBIE'].where(file_mask_basins['ID_IMBIE'] != 1, 159).where(file_msk == 1)
prelim_file = prelim_file.where(file_msk != 2, 0).where(file_msk != 0, 1)
prelim_file = prelim_file.where(prelim_file != 104, 103) # FRIS
prelim_file = prelim_file.where(prelim_file != 58, 57) # Ross

In [ ]:
#whole_ds = isfmf.create_mask_and_metadata_isf(file_msk, -1*file_bed_orig, file_msk, -1*file_draft, file_isf_conc, False, 
#                                          inputpath_metadata+'lonlat_masks.txt', outputpath_mask, 
#                                          inputpath_metadata + 'iceshelves_metadata_Nico.txt', 
#                                          inputpath_metadata+'GL_flux_rignot13.csv', mouginot_basins=True, variable_geometry=False,
#                                          write_ismask = 'yes', write_groundmask = 'yes', write_outfile='yes',
#                                          ground_point ='no',dist=40, add_fac=120, connectivity=4, threshold=4,
#                                          write_metadata='yes')

whole_ds = isfmf.create_mask_and_metadata_isf(file_msk, -1*file_bed_orig, file_msk, -1*file_draft, file_isf_conc, False, 
                                          inputpath_data + 'Mask_Iceshelf_4km_IMBIE_withNisf.nc', outputpath_mask, 
                                          inputpath_metadata + 'iceshelves_metadata_Nico.txt', 
                                          inputpath_metadata+'GL_flux_rignot13.csv', mouginot_basins=True, variable_geometry=False,
                                          write_ismask = 'yes', write_groundmask = 'yes', write_outfile='yes',
                                          ground_point ='no',dist=40, add_fac=250, connectivity=4, threshold=4,
                                          write_metadata='yes')

# Write to netcdf
print('------- WRITE TO NETCDF -----------')
whole_ds.to_netcdf(outputpath_mask + 'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc','w')


In [ ]:
file_bed_orig = -1*file_mask_orig_cut['bed']
file_draft = (file_mask_orig_cut['thickness'] - file_mask_orig_cut['surface']).where(file_msk==1)
file_draft = file_draft.where(file_draft > 0, 0)
file_isf_conc = file_mask_orig_cut['isf_conc']
file_msk = file_mask_orig_cut['mask_0_1_2']  #0 = ocean, 1 = ice shelves, 2 = grounded ice

xx = file_mask_orig_cut['x']
yy = file_mask_orig_cut['y']

whole_ds_tt = xr.open_dataset(outputpath_mask + 'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')

nonnan_Nisf = whole_ds_tt['Nisf'].where(np.isfinite(whole_ds_tt['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = whole_ds_tt.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
isf_var_of_int = whole_ds_tt[['ISF_mask', 'GL_mask', 'dGL', 'dIF', 'latitude', 'longitude', 'isf_name']]
out_2D, out_1D = bf.box_charac_file(file_isf['Nisf'],isf_var_of_int, -1*file_draft, file_isf_conc, outputpath_boxes, max_nb_box=10)
out_2D.to_netcdf(outputpath_boxes + 'BedMachine_4km_boxes_2D_oneFRIS.nc')
out_1D.to_netcdf(outputpath_boxes + 'BedMachine_4km_boxes_1D_oneFRIS.nc')
    
# double check here which ice shelves we want    

In [ ]:
plume_param_options = ['new_lazero'] #'cavity',, 'local','lazero'

plume_var_of_int = file_isf[['ISF_mask', 'GL_mask', 'IF_mask', 'dIF', 'dGL_dIF', 'latitude', 'longitude', 'front_ice_depth_avg']]

# Compute the ice draft
ice_draft_pos = file_draft
# Be careful with ice shelf 178 and 195 - they have a negative ice draft
# I don't know how to fix it at the moment so I put it to nan
#ice_draft_pos = ice_draft_pos.where(plume_var_of_int['ISF_mask'] != 178, np.nan)
#ice_draft_pos = ice_draft_pos.where(plume_var_of_int['ISF_mask'] != 195, np.nan)

ice_draft_neg = -1*ice_draft_pos

plume_charac_shift2 = pf.prepare_plume_charac(plume_param_options, ice_draft_pos, plume_var_of_int, grad_corr=5, dir_nb=16, extra_shift=2, dist_incl=1)
plume_charac_shift1 = pf.prepare_plume_charac(plume_param_options, ice_draft_pos, plume_var_of_int, grad_corr=5, dir_nb=16, extra_shift=1, dist_incl=1)
plume_charac = plume_charac_shift2.combine_first(plume_charac_shift1)
print('------ WRITE TO NETCDF -------')
#plume_charac.to_netcdf(outputpath_plumes+'BedMachine_4km_plume_characteristics.nc')
plume_charac.to_netcdf(outputpath_plumes+'BedMachine_4km_plume_characteristics_lazero_comparison_mixedshift.nc')
# smoothed was with grad_corr=2, dir_nb=16, extra_shift=2, dist_incl=1

In [ ]:
plume_param_options = ['new_lazero'] #'cavity',, 'local','lazero'

plume_var_of_int = file_isf[['ISF_mask', 'GL_mask', 'IF_mask', 'dIF', 'dGL_dIF', 'latitude', 'longitude', 'front_ice_depth_avg']]

# Compute the ice draft
ice_draft_pos = file_draft
# Be careful with ice shelf 178 and 195 - they have a negative ice draft
# I don't know how to fix it at the moment so I put it to nan
#ice_draft_pos = ice_draft_pos.where(plume_var_of_int['ISF_mask'] != 178, np.nan)
#ice_draft_pos = ice_draft_pos.where(plume_var_of_int['ISF_mask'] != 195, np.nan)

ice_draft_neg = -1*ice_draft_pos

plume_charac = pf.prepare_plume_charac(plume_param_options, ice_draft_pos, plume_var_of_int, grad_corr=5, dir_nb=16, extra_shift=1, dist_incl=1)
print('------ WRITE TO NETCDF -------')
#plume_charac.to_netcdf(outputpath_plumes+'BedMachine_4km_plume_characteristics.nc')
plume_charac.to_netcdf(outputpath_plumes+'BedMachine_4km_plume_characteristics_lazero_comparison_shift1.nc')
# smoothed was with grad_corr=2, dir_nb=16, extra_shift=2, dist_incl=1

In [ ]:
plume_charac.to_netcdf(outputpath_plumes+'BedMachine_4km_plume_characteristics_lazero_comparison.nc')


In [ ]:
plume_var_of_int.Nisf

In [ ]:
plume_charac = xr.open_dataset(outputpath_plumes+'BedMachine_4km_plume_characteristics_lazero_comparison.nc')
plume_charac_smoothed = xr.open_dataset(outputpath_plumes+'BedMachine_4km_plume_characteristics_lazero_comparison_smoothed.nc')

In [ ]:
alpha_test = plume_charac['alpha'].sel(option='new_lazero')
#alpha_test_smoothed = plume_charac_smoothed['alpha'].sel(option='new_lazero')

In [ ]:
zGL_test = plume_charac['zGL'].sel(option='new_lazero')
#zGL_test_smoothed = plume_charac_smoothed['zGL'].sel(option='new_lazero')

In [ ]:
zGL_test.plot()

In [ ]:
np.isnan(zGL_test).sum()

In [ ]:
zGL_test.where(file_isf['ISF_mask'] > 1).plot()

In [ ]:
alpha_test.where(file_isf['ISF_mask'] > 1).plot()

In [ ]:
plt.figure()
(zGL_test_smoothed - zGL_test).plot(vmax=0.005)

In [ ]:
plt.figure()
(alpha_test_smoothed - alpha_test).plot(vmax=0.005)

In [ ]:
zGL_test.where((zGL_test == 0) & (file_isf['ISF_mask'] > 1)).plot()

In [ ]:
(np.isnan(alpha_test) & (file_isf['ISF_mask'] > 1)).plot()

In [ ]:
plt.figure()
alpha_test_smoothed.where((alpha_test_smoothed == 0) & (file_isf['ISF_mask'] > 1)).plot()

In [ ]:
np.tan(plume_charac['alpha'].sel(option='new_lazero')).plot(vmax=0.005,cmap=mpl.cm.jet)

In [ ]:
for kisf in file_isf.Nisf.isel(Nisf=range(10)):
    plt.figure()
    ((plume_charac['alpha'].sel(option='new_lazero') == 0).where(file_isf['ISF_mask'] == kisf, drop=True)).plot()
    plt.title(str(kisf.values)+' '+str(file_isf['isf_name'].sel(Nisf=kisf).values))

In [ ]:
file_isf['ISF_mask'].plot()

In [ ]:
(plume_charac['alpha'].where((file_isf['ISF_mask'] > 5) & (file_isf['ISF_mask'] < 500)).sel(option='new_lazero') == 0).plot()

In [ ]:
plume_charac.to_netcdf(outputpath_plumes+'BedMachine_4km_plume_characteristics.nc')

In [ ]:
plume_charac = xr.open_dataset(outputpath_plumes+'BedMachine_4km_plume_characteristics.nc')

In [ ]:
file_isf['isf_name'].sel(Nisf=42)

In [ ]:
alpha_kisf = plume_charac['alpha'].sel(option='lazero').where(file_isf['ISF_mask'] == 42, drop=True)

In [ ]:
alpha_kisf.where(alpha_kisf > 0).plot()

In [ ]:
plume_charac['alpha'].sel(option='lazero').where(plume_charac['alpha'].sel(option='lazero') == 0).plot()

FILL POINTS WHERE SLOPE IS NOT INCLINED TOWARDS GROUNDING LINE

In [ ]:
plume_charac = xr.open_dataset(outputpath_plumes+'BedMachine_4km_plume_characteristics.nc')

alpha_all_corrected = plume_charac['alpha'].sel(option='lazero') * np.nan
zGL_all_corrected = plume_charac['zGL'].sel(option='lazero') * np.nan

for kisf in tqdm(file_isf.Nisf):
    alpha_kisf_lazero = plume_charac['alpha'].sel(option='lazero').where(file_isf['ISF_mask']==kisf, drop=True).drop('Nisf')
    alpha_kisf_local = plume_charac['alpha'].sel(option='local').where(file_isf['ISF_mask']==kisf, drop=True).drop('Nisf')
    alpha_kisf_lazero_corrected = alpha_kisf_lazero.where(alpha_kisf_lazero > 0, alpha_kisf_local)

    zGL_kisf_lazero = plume_charac['zGL'].sel(option='lazero').where(file_isf['ISF_mask']==kisf, drop=True).drop('Nisf')
    zGL_kisf_local = plume_charac['zGL'].sel(option='local').where(file_isf['ISF_mask']==kisf, drop=True).drop('Nisf')

    zGL_kisf_lazero_fillx = zGL_kisf_lazero.where(alpha_kisf_lazero != 0).interpolate_na(dim='x',method='nearest')
    zGL_kisf_lazero_filly = zGL_kisf_lazero.where(alpha_kisf_lazero != 0).interpolate_na(dim='y',method='nearest')
    zGL_kisf_lazero_maxboth = xr.concat([zGL_kisf_lazero_fillx,zGL_kisf_lazero_filly],dim='dim0').max('dim0')
    #zGL_kisf_lazero_999_minboth = xr.concat([zGL_kisf_lazero_999_fillx,zGL_kisf_lazero_999_filly],dim='dim0').mean('dim0')
    zGL_kisf_lazero_corrected = zGL_kisf_lazero_maxboth.where(np.isfinite(zGL_kisf_lazero))
    
    alpha_all_corrected = alpha_all_corrected.where(file_isf['ISF_mask']!=kisf, alpha_kisf_lazero_corrected.reindex_like(alpha_all_corrected))
    zGL_all_corrected = zGL_all_corrected.where(file_isf['ISF_mask']!=kisf, zGL_kisf_lazero_corrected.reindex_like(alpha_all_corrected))
alpha_all_corrected = alpha_all_corrected.where(np.isfinite(alpha_all_corrected), 0)
zGL_all_corrected = zGL_all_corrected.where(np.isfinite(alpha_all_corrected), 0)

In [ ]:
plume_charac['alpha'] = xr.concat([plume_charac['alpha'].sel(option='cavity'),alpha_all_corrected,plume_charac['alpha'].sel(option='local')], dim='option', coords='minimal')
plume_charac['zGL'] = xr.concat([plume_charac['zGL'].sel(option='cavity'),zGL_all_corrected,plume_charac['zGL'].sel(option='local')], dim='option', coords='minimal')


In [ ]:
print('------ WRITE TO NETCDF -------')
plume_charac.to_netcdf(outputpath_plumes+'BedMachine_4km_plume_characteristics_corrected.nc')

In [ ]:
plt.figure()
plume_charac['zGL'].sel(option='lazero').plot()

In [ ]:
plt.figure()
(plume_charac['alpha'].sel(option='local') == 0).plot()

In [ ]:
plume_charac_orig = xr.open_dataset(outputpath_plumes+'BedMachine_4km_plume_characteristics.nc')
plume_charac_corr = xr.open_dataset(outputpath_plumes+'BedMachine_4km_plume_characteristic_corrected.nc')

In [ ]:
kisf = 66
plt.figure()
plume_charac_orig['zGL'].sel(option='lazero').where(whole_ds_tt['ISF_mask'] == kisf, drop=True).plot()
plt.figure()
plume_charac_corr['zGL'].sel(option='lazero').where(whole_ds_tt['ISF_mask'] == kisf, drop=True).plot()
plt.figure()
plume_charac_orig['alpha'].sel(option='lazero').where(whole_ds_tt['ISF_mask'] == kisf, drop=True).plot()
plt.figure()
plume_charac_corr['alpha'].sel(option='lazero').where(whole_ds_tt['ISF_mask'] == kisf, drop=True).plot()

DO THE MASKS FOR 8 KM GRID AS WELL

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
inputpath_metadata='/bettik/burgardc/SCRIPTS/basal_melt_param/data/raw/MASK_METADATA/'
outputpath_mask ='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_8km/'


file_mask_orig = xr.open_dataset(inputpath_data+'BedMachine_v3_aggregated8km_allvars.nc')
file_mask_orig_cut = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim)

In [ ]:
file_msk = file_mask_orig_cut['mask_0_1_2']  #0 = ocean, 1 = ice shelves, 2 = grounded ice

In [ ]:
file_bed_orig = -1*file_mask_orig_cut['bed']
file_draft = (file_mask_orig_cut['thickness'] - file_mask_orig_cut['surface']).where(file_msk==1)
file_isf_conc = file_mask_orig_cut['isf_conc']

xx = file_msk['x']
yy = file_msk['y']

dx = abs(xx[1] - xx[0])
dy = abs(yy[1] - yy[0])

In [ ]:
whole_ds = isfmf.create_mask_and_metadata_isf(file_msk, -1*file_bed_orig, file_msk, -1*file_draft, file_isf_conc, False, 
                                          inputpath_metadata+'lonlat_masks.txt', outputpath_mask, 
                                          inputpath_metadata + 'iceshelves_metadata_Nico.txt', 
                                          inputpath_metadata+'GL_flux_rignot13.csv', mouginot_basins=False, variable_geometry=False,
                                          write_ismask = 'yes', write_groundmask = 'yes', write_outfile='yes',
                                          ground_point ='no',dist=40, add_fac=120, connectivity=4, threshold=4,
                                          write_metadata='yes')

# Write to netcdf
print('------- WRITE TO NETCDF -----------')
whole_ds.to_netcdf(outputpath_mask + 'BedMachine_8km_isf_masks_and_info_and_distance_oneFRIS.nc','w')
